## Analyzing National Public Radio Articles

Figure out clusters of ~12k articles from NPR (National Public Radio) through the use of LDA.
<br><br>The data was obtained from their website [www.npr.org](http://www.npr.org).

In [1]:
import pandas as pd

In [2]:
npr = pd.read_csv('npr.csv')

In [3]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [4]:
len(npr)

11992

## Preprocessing

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [6]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [7]:
dtm = cv.fit_transform(npr['Article'])

In [8]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

This means the document term matrix has 5477 unique features and 11992 words

## LDA

In [9]:
from sklearn.decomposition import LatentDirichletAllocation

In [10]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [11]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words

In [12]:
len(cv.get_feature_names())

54777

In [13]:
type(cv.get_feature_names())

list

In [14]:
import random

random_word_id=random.randint(0,54777)
cv.get_feature_names()[random_word_id]

'fp'

This just randomly displays any of the feature words

### Showing Top Words Per Topic

In [15]:
len(LDA.components_)

7

In [16]:
main_topic =LDA.components_[0]

In [17]:
main_topic.argsort()

array([ 2475, 18302, 35285, ..., 22673, 42561, 42993], dtype=int64)

In [18]:
#ARGS0RT ---> INDEX POSITIONS SORTED FROM LEAST TO GREATEST
#TOP 10 VALUES (HIGHEST) = lAST 10 VALUES OF ARGSORT

main_topic.argsort()[-10:]

array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993], dtype=int64)

In [19]:
top_ten_words =main_topic.argsort()[-10:]

In [45]:
#To display the top 10 words

for i in top_ten_words:
    print (cv.get_feature_names()[i])

new
percent
government
company
million
care
people
health
said
says


In [21]:
#This is to display the top 15 words per topic
for i,topic in enumerate(LDA.components_):
    print(f"The TOP 15 words for Topic #{i}")
    print([cv.get_feature_names()[index] for index in topic.argsort()[-15:]])
    print ('\n')

The TOP 15 words for Topic #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


The TOP 15 words for Topic #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


The TOP 15 words for Topic #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


The TOP 15 words for Topic #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


The TOP 15 words for Topic #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


The TOP 15 words for Topic #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

### Attaching Discovered Topic Labels to Original Articles

In [22]:
topic_results = LDA.transform(dtm)

In [23]:
topic_results[0].round(2)


array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [24]:
topic_results[0].argmax()

1

### Combining with Original Data

In [36]:
npr['Topic']=topic_results.argmax(axis=1)

In [43]:
npr

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
...,...,...
11987,The number of law enforcement officers shot an...,1
11988,"Trump is busy these days with victory tours,...",4
11989,It’s always interesting for the Goats and Soda...,3
11990,The election of Donald Trump was a surprise to...,4


In [41]:
npr.head(n=20)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2


Below are the potential topics/clusters based on the top 15 features/words generated above. <br><br>
It turns out that there are some overlapping topics. It may be worth expanding the number of features/words per topic to get a clearer sense of each topic or reducing the number of components.
<br><br>
Topic 0 - [unclear] <br>
Topic 1 - government<br>
Topic 2 - [unclear]<br>
Topic 3 - health<br>
Topic 4 - election<br>
Topic 5 - [unclear]<br>
Topic 6 - Education<br>